In [4]:
"""
DISCLAIMER: 

This software is provided solely for educational and research purposes. 
It is not intended to provide investment advice, and no investment recommendations are made herein. 
The developers are not financial advisors and accept no responsibility for any financial decisions or losses resulting from the use of this software. 
Always consult a professional financial advisor before making any investment decisions.
"""

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.interpolate import interp1d

# --- Functions for Calculations ---

def yang_zhang(price_data, window=30, trading_periods=252):
    """
    Calculate realized volatility using the Yang–Zhang method.
    price_data must contain the columns: 'Open', 'High', 'Low', 'Close'
    """
    log_ho = np.log(price_data['High'] / price_data['Open'])
    log_lo = np.log(price_data['Low'] / price_data['Open'])
    log_co = np.log(price_data['Close'] / price_data['Open'])
    
    log_oc = np.log(price_data['Open'] / price_data['Close'].shift(1))
    log_oc_sq = log_oc ** 2
    
    log_cc = np.log(price_data['Close'] / price_data['Close'].shift(1))
    log_cc_sq = log_cc ** 2
    
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    
    close_vol = log_cc_sq.rolling(window=window).sum() / (window - 1.0)
    open_vol  = log_oc_sq.rolling(window=window).sum() / (window - 1.0)
    window_rs = rs.rolling(window=window).sum() / (window - 1.0)
    
    k = 0.34 / (1.34 + ((window + 1) / (window - 1)))
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * np.sqrt(trading_periods)
    return result.iloc[-1]

def build_term_structure(days, ivs):
    """
    Build a linear spline term structure from days-to-expiry and corresponding ATM IV values.
    Returns a function that interpolates IV for a given day.
    """
    days = np.array(days)
    ivs = np.array(ivs)
    sort_idx = days.argsort()
    days = days[sort_idx]
    ivs = ivs[sort_idx]
    spline = interp1d(days, ivs, kind='linear', fill_value="extrapolate")
    
    def term_spline(dte):
        if dte < days[0]:
            return ivs[0]
        elif dte > days[-1]:
            return ivs[-1]
        else:
            return float(spline(dte))
    return term_spline

# --- Helper Functions ---

def get_atm_iv_for_exp(stock, exp_str):
    """
    Retrieve the ATM implied volatility for a given expiration date.
    """
    try:
        chain = stock.option_chain(exp_str)
    except Exception:
        return None
    calls = chain.calls
    puts = chain.puts
    if calls.empty or puts.empty:
        return None
    underlying_price = get_current_price(stock)
    # Find the call and put with strike closest to the underlying price.
    calls['abs_diff'] = (calls['strike'] - underlying_price).abs()
    puts['abs_diff'] = (puts['strike'] - underlying_price).abs()
    call_idx = calls['abs_diff'].idxmin()
    put_idx = puts['abs_diff'].idxmin()
    call_iv = calls.loc[call_idx, 'impliedVolatility']
    put_iv = puts.loc[put_idx, 'impliedVolatility']
    atm_iv = (call_iv + put_iv) / 2.0
    return atm_iv

def get_sp500_tickers():
    """
    Scrape S&P 500 tickers from Wikipedia.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    tickers = sp500_table['Symbol'].tolist()
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    return tickers

def get_current_price(stock):
    """
    Retrieve the current (latest available) closing price for the given yfinance ticker.
    """
    todays_data = stock.history(period='1d')
    if todays_data.empty:
        raise ValueError("No market price found.")
    return todays_data['Close'].iloc[0]

# --- Core Screening Function ---

def compute_metrics(ticker_symbol):
    """
    For a given ticker symbol, compute the following metrics:
      - avg_volume: whether the 30-day moving average volume is >= 1.5M shares.
      - iv30_rv30: whether the ratio of the 30-day ATM IV (closest to 30 days) to realized volatility (Yang-Zhang) is >= 1.25.
      - ts_slope: whether the term structure slope between the nearest expiring option and the nearest one expiring between 20-45 days is <= -0.00406.
      - expected_move: the straddle-based expected move (as a percentage string), computed from the near-term options chain.
    
    Returns a dictionary with boolean flags (pass/fail) for each metric plus the expected move.
    Returns None if data is incomplete.
    """
    try:
        ticker_symbol = ticker_symbol.strip().upper()
        if not ticker_symbol:
            return None
        
        stock = yf.Ticker(ticker_symbol)
        if len(stock.options) == 0:
            return None

        # Get all available expiration dates from yfinance (as date objects)
        all_exps = list(stock.options)
        if not all_exps:
            return None
        today = datetime.today().date()
        all_exps_dates = sorted(datetime.strptime(d, "%Y-%m-%d").date() for d in all_exps)
        
        # Select the nearest expiring option (first_expiry)
        first_expiry = all_exps_dates[0]
        first_exp_str = first_expiry.strftime("%Y-%m-%d")
        
        # Select the nearest option expiring between 20 and 45 days out.
        filtered_exps = [d for d in all_exps_dates if 20 <= (d - today).days <= 45]
        if filtered_exps:
            second_expiry = filtered_exps[0]
            second_exp_str = second_expiry.strftime("%Y-%m-%d")
        else:
            second_expiry = None
            second_exp_str = None
        
        # For the iv30/rv30 ratio, pick the expiration closest to 30 days.
        target30 = min(all_exps_dates, key=lambda d: abs((d - today).days - 30))
        target30_str = target30.strftime("%Y-%m-%d")
        
        # Get the ATM IV for each chosen expiration.
        iv_first = get_atm_iv_for_exp(stock, first_exp_str)
        iv_target30 = get_atm_iv_for_exp(stock, target30_str)
        if iv_first is None or iv_target30 is None:
            return None
        if second_expiry is not None:
            iv_second = get_atm_iv_for_exp(stock, second_exp_str)
        else:
            iv_second = None
        
        dte_first = (first_expiry - today).days
        dte_target30 = (target30 - today).days
        if second_expiry is not None:
            dte_second = (second_expiry - today).days
            ts_slope = (iv_second - iv_first) / (dte_second - dte_first)
        else:
            ts_slope = None  # Unable to compute slope without a second expiration
        
        # Calculate iv30/rv30 ratio using 3 months of price history.
        price_history = stock.history(period='3mo')
        if price_history.empty:
            return None
        rv30 = yang_zhang(price_history)
        if rv30 == 0:
            return None
        iv30_rv30 = iv_target30 / rv30
        
        # Compute the 30-day moving average volume.
        avg_volume = price_history['Volume'].rolling(window=30).mean().dropna().iloc[-1]
        
        # For expected move, use the near-term (first_expiry) options chain.
        try:
            chain_first = stock.option_chain(first_exp_str)
        except Exception:
            return None
        calls = chain_first.calls
        puts = chain_first.puts
        if calls.empty or puts.empty:
            return None
        calls['abs_diff'] = (calls['strike'] - get_current_price(stock)).abs()
        puts['abs_diff'] = (puts['strike'] - get_current_price(stock)).abs()
        call_idx = calls['abs_diff'].idxmin()
        put_idx = puts['abs_diff'].idxmin()
        call_bid = calls.loc[call_idx, 'bid']
        call_ask = calls.loc[call_idx, 'ask']
        put_bid = puts.loc[put_idx, 'bid']
        put_ask = puts.loc[put_idx, 'ask']
        if call_bid is not None and call_ask is not None:
            call_mid = (call_bid + call_ask) / 2.0
        else:
            call_mid = None
        if put_bid is not None and put_ask is not None:
            put_mid = (put_bid + put_ask) / 2.0
        else:
            put_mid = None
        if call_mid is None or put_mid is None:
            return None
        straddle = call_mid + put_mid
        underlying_price = get_current_price(stock)
        expected_move = f"{round(straddle / underlying_price * 100, 2)}%"
        
        return {
            'avg_volume': avg_volume >= 1500000,
            'iv30_rv30': iv30_rv30 >= 1.25,
            # If we couldn’t compute slope, mark it as failed.
            'ts_slope': ts_slope is not None and ts_slope <= -0.00406,
            'expected_move': expected_move,
            'raw': {
                'avg_volume': avg_volume,
                'iv30_rv30': iv30_rv30,
                'ts_slope': ts_slope if ts_slope is not None else 0,
                'iv_target30': iv_target30,
                'iv_first': iv_first,
                'iv_second': iv_second if iv_second is not None else 0,
                'dte_first': dte_first,
                'dte_second': (dte_second if second_expiry is not None else None),
                'dte_target30': dte_target30,
                'straddle': straddle,
                'underlying_price': underlying_price
            }
        }
    except Exception as e:
        # In production, you would log the error e.
        return None

def classify(metrics):
    """
    Classify a ticker based on its metrics.
      - Recommended: All three metrics pass.
      - Consider: Term slope passes and only one of the other two passes.
      - Avoid: Otherwise.
    """
    avg_volume_pass = metrics['avg_volume']
    iv30_rv30_pass  = metrics['iv30_rv30']
    ts_slope_pass   = metrics['ts_slope']
    
    if avg_volume_pass and iv30_rv30_pass and ts_slope_pass:
        return "Recommended"
    elif ts_slope_pass and ((avg_volume_pass and not iv30_rv30_pass) or (iv30_rv30_pass and not avg_volume_pass)):
        return "Consider"
    else:
        return "Avoid"

# --- Main Screening Script ---

def main():
    sp500_tickers = get_sp500_tickers()
    results = {}  # To hold classification and metrics for each ticker
    
    print("Screening S&P 500 companies based on all three metrics...\n")
    
    for ticker in sp500_tickers:
        print(f"Processing {ticker}...")
        metrics = compute_metrics(ticker)
        if metrics is None:
            print(f"  Data unavailable or incomplete for {ticker}. Skipping.\n")
            continue
        
        classification = classify(metrics)
        results[ticker] = {
            'classification': classification,
            'metrics': metrics
        }
        
        # Print the results for this ticker immediately:
        print(f"Ticker: {ticker}")
        print(f"  Classification: {classification}")
        print(f"  avg_volume >= 1.5M: {'PASS' if metrics['avg_volume'] else 'FAIL'} (30-day MA = {metrics['raw']['avg_volume']:.0f})")
        print(f"  iv30/rv30 >= 1.25: {'PASS' if metrics['iv30_rv30'] else 'FAIL'} (iv30/rv30 = {metrics['raw']['iv30_rv30']:.2f})")
        if metrics['raw']['dte_second'] is not None:
            print(f"  ts_slope (Nearest vs. 20-45 days): {'PASS' if metrics['ts_slope'] else 'FAIL'} (Slope = {metrics['raw']['ts_slope']:.5f})")
        else:
            print("  ts_slope: Unable to compute (no expiration found between 20-45 days)")
        print(f"  Expected Move: {metrics['expected_move']}\n")
    
    # Final summary:
    if not results:
        print("No tickers met the data requirements for screening.")
    else:
        print("\nFinal Screening Results:")
        for ticker, result in results.items():
            print(f"Ticker: {ticker}  ->  Classification: {result['classification']}")
    
if __name__ == "__main__":
    main()


Screening S&P 500 companies based on all three metrics...

Processing MMM...
Ticker: MMM
  Classification: Avoid
  avg_volume >= 1.5M: PASS (30-day MA = 4599210)
  iv30/rv30 >= 1.25: FAIL (iv30/rv30 = 0.95)
  ts_slope (Nearest vs. 20-45 days): FAIL (Slope = -0.00051)
  Expected Move: 3.43%

Processing AOS...
Ticker: AOS
  Classification: Avoid
  avg_volume >= 1.5M: PASS (30-day MA = 1565120)
  iv30/rv30 >= 1.25: FAIL (iv30/rv30 = 1.12)
  ts_slope (Nearest vs. 20-45 days): FAIL (Slope = -0.00229)
  Expected Move: 2.91%

Processing ABT...
Ticker: ABT
  Classification: Consider
  avg_volume >= 1.5M: PASS (30-day MA = 7786040)
  iv30/rv30 >= 1.25: FAIL (iv30/rv30 = 0.96)
  ts_slope (Nearest vs. 20-45 days): PASS (Slope = -0.01107)
  Expected Move: 4.39%

Processing ABBV...
Ticker: ABBV
  Classification: Avoid
  avg_volume >= 1.5M: PASS (30-day MA = 8458377)
  iv30/rv30 >= 1.25: FAIL (iv30/rv30 = 0.87)
  ts_slope (Nearest vs. 20-45 days): FAIL (Slope = -0.00282)
  Expected Move: 3.24%

Proc

KeyboardInterrupt: 